In [7]:
import os

In [16]:
%pwd

'e:\\Python Projects\\Deep_learning\\mlops_test1'

In [9]:
os.chdir("../")

In [10]:
# In order to get specific entity ( object with data + configuration variables )
# We need to create a special dataclass

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig :
  root_directory: Path
  # getting dataset from raw github
  source_URL: str
  local_data_file: Path
  unzip_dir:  Path

In [12]:
# Lets import constants
from src.chicken_disease_classification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.chicken_disease_classification.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self):
        self.config_file_path= CONFIG_FILE_PATH
        self.params_file_path= PARAMS_FILE_PATH

        self.config= read_yaml(CONFIG_FILE_PATH)
        self.params= read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_configuration(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_directory])

        data_ingestion_config= DataIngestionConfig(root_directory= config.root_directory \
                                                   , source_URL= config.source_URL \
                                                   , local_data_file= config.local_data_file\
                                                   ,unzip_dir= config.unzip_dir
                                                    )
        return data_ingestion_config




In [14]:
import os
from urllib import request
import zipfile
from src.chicken_disease_classification import logger
from src.chicken_disease_classification.utils.common import get_size

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config= config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename,headers= request.urlretrieve(
                    url= self.config.source_URL,
                    filename= self.config.local_data_file
                )
                logger.info(f"filename:{filename} downloaded with the following info  :File location: {os.path.abspath(filename)} \n {headers}")
            except Exception as e:
                logger.error(f"filename: {filename} download failed with Exception {e}")
        else:
             logger.info(f"filename {self.config.local_data_file} already exists with size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip(self):
        """
        Extracts all files from a zip archive.

        Args:
            zip_file_path (str): The path to the zip file.
            extract_path (str, optional): The directory to extract the files to.
                Defaults to the current directory (".")
        """
        zip_file_path, extract_path= self.config.local_data_file,self.config.unzip_dir
        
        try:
        
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)
            logger.info(f"Successfully extracted all files from '{zip_file_path}' to '{extract_path}'")
        except zipfile.BadZipFile:
            logger.error(f"Error: '{zip_file_path}' is not a valid zip file.")
        except FileNotFoundError:
            logger.error(f"Error: File not found at '{zip_file_path}'")
        except Exception as e:
            logger.error(f"An unexpected error occurred: {e}")
            logger.debug("Exception details:", exc_info=True)  # Log detailed traceback

            




In [15]:
# Lets get the pipeline
try:
    configuration_manager= ConfigurationManager()
    data_ingestion_config_entity= configuration_manager.get_data_ingestion_configuration()
    data_ingester_component= DataIngestion(config= data_ingestion_config_entity)
    data_ingester_component.download_file()
    data_ingester_component.extract_zip()
except Exception as e:
    logger.error("Data Ingestion Setup failed with Exception: {e}")
    raise e


2025-05-15 17:35:48,397 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\config\config.yaml loaded successfully
2025-05-15 17:35:48,401 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\params.yaml loaded successfully
2025-05-15 17:35:48,401 - INFO - created directory at: artifacts
2025-05-15 17:35:48,401 - INFO - created directory at: artifacts/data_ingestion
2025-05-15 17:35:53,143 - INFO - filename:artifacts/data_ingestion/data.zip downloaded with the following info  :File location: e:\Python Projects\Deep_learning\mlops_test1\artifacts\data_ingestion\data.zip 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Reque